In [ ]:
import pandas as pd
import pypsa

# Load network to modify
n = pypsa.Network(snakemake.input["network"])

# Load technology data
df = pd.read_csv(snakemake.input["technologydata"], dtype={"value": float})

df = df.set_index("name")


def calculate_annuity_factor(lifetime, rate):
    if rate == 0.0:
        return 1 / lifetime
    elif lifetime == 0.0:
        return 0
    else:
        return rate / (1.0 - 1.0 / ((1.0 + rate) ** lifetime))


discount_rate = (
    float(
        df.query("name == 'discount rate' and parameter == 'discount rate'").iloc[0][
            "value"
        ]
    )
    / 100
)

lifetimes = df.query("parameter == 'lifetime'")["value"]
annuity_factors = lifetimes.apply(calculate_annuity_factor, rate=discount_rate)
annual_investment_cost = (
    df.query("parameter == 'investment'")["value"] * annuity_factors
    + df.query("parameter == 'FOM'")["value"]
)

# Take number of years the model spans into account
# to determine capital_cost from annual cost
number_years = len(n.snapshots.year.unique())
capital_cost = annual_investment_cost * number_years
capital_cost = capital_cost.fillna(0.0)
capital_cost.name = "capital_cost"

marginal_cost = (
    df.query("parameter == 'fuel cost'")["value"]
    + df.query("parameter == 'VOM'")["value"]
)
marginal_cost /= 1000  # unit is kWh not MWh
marginal_cost.name = "marginal_cost"

# Attach costs and other technology properties to network components
for idx, component in n.links.iterrows():
    efficiency = df.loc[idx].query("parameter=='efficiency'").iloc[0]["value"]
    n.links.loc[idx, "efficiency"] = efficiency

    # cost are relative to bus1, but need to be relative to bus0 for pypsa -> scale with efficiency(bus0, bus1)
    n.links.loc[idx, "capital_cost"] = capital_cost.loc[idx] * efficiency
    n.links.loc[idx, "marginal_cost"] = marginal_cost.loc[idx] * efficiency

for idx, component in n.generators.iterrows():
    efficiency = df.loc[idx].query("parameter=='efficiency'").iloc[0]["value"]
    n.generators.loc[idx, "efficiency"] = efficiency

    n.generators.loc[idx, "capital_cost"] = capital_cost.loc[idx]
    n.generators.loc[idx, "marginal_cost"] = marginal_cost.loc[idx]

for idx, component in n.stores.iterrows():
    n.stores.loc[idx, "capital_cost"] = capital_cost.loc[idx]
    n.stores.loc[idx, "marginal_cost"] = marginal_cost.loc[idx]

    n.stores.loc[idx, "standing_loss"] = (
        df.loc[idx].query("parameter=='standing losses'").iloc[0]["value"]
    )

# Save modified network
n.export_to_netcdf(snakemake.output["network"])